# CPX Code
Load the following into `code.py`:
```python
# circuitpython firmware
import time
import touchio
import board

touch_a1 = touchio.TouchIn(board.A1)

def command_TOUCH1():
    v = touch_a1.raw_value
    print("%d" % (v,))

commands = {
  'TOUCH1': command_TOUCH1,
}

t0 = time.monotonic()
while True:
    t1 = time.monotonic()
    s = input()
    try:
        com_func = commands[s]
        com_func()
    except KeyError:
        print("# ERROR: INVALID COMMAND '%s'" % (s,))
```

# Testing in the notebook
Make sure no other process is accessing the CPX serial port (/dev/ttyACM0 on Linux). Now we can use pyserial to test communication:

In [1]:
import glob
import serial
port = glob.glob("/dev/ttyACM*")[-1]
ser = serial.Serial(port,112500)


In [2]:
ser.write("TOUCH1\r\n")
print(ser.readline().strip()) #echoed command
print(ser.readline().strip()) #sensor value

TOUCH1
1505


In [3]:
def read_touch1():
    ser.write("TOUCH1\r\n")
    ser.readline() #discard echoed command
    return int(ser.readline().strip())  #send back value as integer

In [2]:
read_touch1()

NameError: name 'read_touch1' is not defined

# What are the problems?




- **MAJOR:** The function call `input()` waits until we press `<enter>` key, so we cannot do anything else in the meantime!  It is a "blocking read". For example, what if we want to blink the LED?
 - *Note* This is a limitation of the current CircuitPython Firmware, we have provided a modified firmware that gets around the problem!


- *MINOR:* The command as written is not very smart.  Do we need to write a new command to enable the other 5 touch sensing inputs? 

- *MINOR:* The command is hard-coded to use the `print` function.  What if we want to send the output somewhere else, for example the UART interface or a network interface?